# Lab: DataFrame API and Spark SQL


## Social characteristics of the Marvel Universe

In this lab you be working with a dataset that was creted about the Marvel Comics universe. The source data is a text file that was created with a graph analysis library outside of Spark, so the text file contains a lot of information and is not in a format that is easy to query with SQL. You are going to use it in this lab to practice data ingestion, manipulation and analysis using both the DataFrame API and Spark SQL. You can see more about the source data [here](http://bioinfo.uib.es/~joemiro/marvel.html).

## Understanding the data file

As mentioned above, the data file contains information about Marvel characters, publications, and which character appeared in each publication. There are two sections in the file:

- Vertices (section begins in line 1 with a header in the form of `*Vertices 19428 6486`):
    - Characters: lines 2-6487 in the format `6481 "DETHSTRYK"`, where `6481` is the node id and the name is within double quotes
    - Publications: lines 6488-19429 in the same format as characters
- Edgeslist (section begins in line 19430 with a header in the form of `*Edgeslist`): lines 19431 to the end of the file. The edge information is in the following format: `2 6488 6489 6490 6491 6492 6493 6494 6495 6496`. This represents a relationship between the character id (the first number) and the publication id's (all other numbers.)

Type `spark` to see the attributes of your `SparkSession`. The `SparkSession` is your entry point to using the DataFrame API.

In [0]:
spark

Out[1]:

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.2.1 
 Master 
 spark://10.139.64.4:7077 
 AppName 
 Databricks Shell

Type `sc` to see the `SparkContext` attributes. The `SparkContext` is your entry point to the RDDs.

In [0]:
sc

Out[2]:

SparkContext 

 Spark UI 

 
 Version 
 v3.2.1 
 Master 
 spark://10.139.64.4:7077 
 AppName 
 Databricks Shell

## Read in the data.

Although the data is a flat file with the structure explained earlier, you will be working with this file using [Spark DataFrame API and Spark SQL](https://spark.apache.org/docs/latest/sql-programming-guide.html).

In the `00-setup-data-connection` workbook you created a mount point for the cloud location of the data.

Load in the text file using [Generic load functions for SparkSQL](https://spark.apache.org/docs/2.3.0/sql-programming-guide.html#data-sources), and load the file `/mnt/course-datasets/marvel/porgat.txt`

Create a DataFrame called `df_in` with a single field, where each record is the full text per line in the original text file.

In [0]:
path = "/mnt/course-datasets/marvel/porgat.txt"
df_in = spark.read.text(path)

Look at the first few lines of `df_in`. What is the default field name?

In [0]:
df_in.head(10)

Out[4]: [Row(value='*Vertices 19428 6486'),
 Row(value='1 "24-HOUR MAN/EMMANUEL"'),
 Row(value='2 "3-D MAN/CHARLES CHAN"'),
 Row(value='3 "4-D MAN/MERCURIO"'),
 Row(value='4 "8-BALL/"'),
 Row(value='5 "A"'),
 Row(value='6 "A\'YIN"'),
 Row(value='7 "ABBOTT, JACK"'),
 Row(value='8 "ABCISSA"'),
 Row(value='9 "ABEL"')]

Count the number of rows in `df_in`:

In [0]:
number_of_rows = df_in.count()
print(number_of_rows)

30520

What happens to a DataFrame when you bring it into your Python session from the cluster? Get the first few records of the `df_in` and save it into an object in your Python session:

In [0]:
databricks_lab = df_in.head(10)

Explore the local Python object. How is it different than the DataFrame in the cluster? Read up on the [Pyspark Row object](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=row#pyspark.sql.Row)

In [0]:
print(databricks_lab)

[Row(value='*Vertices 19428 6486'), Row(value='1 "24-HOUR MAN/EMMANUEL"'), Row(value='2 "3-D MAN/CHARLES CHAN"'), Row(value='3 "4-D MAN/MERCURIO"'), Row(value='4 "8-BALL/"'), Row(value='5 "A"'), Row(value='6 "A\'YIN"'), Row(value='7 "ABBOTT, JACK"'), Row(value='8 "ABCISSA"'), Row(value='9 "ABEL"')]

In [0]:
df_in.head(10)

Out[13]: [Row(value='*Vertices 19428 6486'),
 Row(value='1 "24-HOUR MAN/EMMANUEL"'),
 Row(value='2 "3-D MAN/CHARLES CHAN"'),
 Row(value='3 "4-D MAN/MERCURIO"'),
 Row(value='4 "8-BALL/"'),
 Row(value='5 "A"'),
 Row(value='6 "A\'YIN"'),
 Row(value='7 "ABBOTT, JACK"'),
 Row(value='8 "ABCISSA"'),
 Row(value='9 "ABEL"')]

## Arrange the data

Since the data is in text format, at the end of this section you will have two DataFrames: one for nodes (both characters and publications) and one for edges (the relationship between characters and publications.)

The DataFrame API and SparkSQL functions are in the [pyspark.sql.functions](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.functions) library. You can import all of the functions or specific functions as needed.

Start by importing the `monotonically_increasing_id` function:

In [0]:
from pyspark.sql.functions import monotonically_increasing_id

Create a new dataframe called `df_in_idx` where you will add a new field called `idx` that uses the `monotonically_increasing_id` to add a unique incremental numeric index to each record. You should read more about the function and understand how it works.

In [0]:
df_in_idx = df_in.withColumn("idx", monotonically_increasing_id())

See your new dataframe, you will see the new column.

In [0]:
df_in_idx.show(10)

+--------------------+---+
 value|idx|
+--------------------+---+
*Vertices 19428 6486| 0|
1 "24-HOUR MAN/EM...| 1|
2 "3-D MAN/CHARLE...| 2|
3 "4-D MAN/MERCURIO"| 3|
 4 "8-BALL/"| 4|
 5 "A"| 5|
 6 "A'YIN"| 6|
 7 "ABBOTT, JACK"| 7|
 8 "ABCISSA"| 8|
 9 "ABEL"| 9|
+--------------------+---+
only showing top 10 rows

In [0]:
df_in_idx.head(10)

Out[18]: [Row(value='*Vertices 19428 6486', idx=0),
 Row(value='1 "24-HOUR MAN/EMMANUEL"', idx=1),
 Row(value='2 "3-D MAN/CHARLES CHAN"', idx=2),
 Row(value='3 "4-D MAN/MERCURIO"', idx=3),
 Row(value='4 "8-BALL/"', idx=4),
 Row(value='5 "A"', idx=5),
 Row(value='6 "A\'YIN"', idx=6),
 Row(value='7 "ABBOTT, JACK"', idx=7),
 Row(value='8 "ABCISSA"', idx=8),
 Row(value='9 "ABEL"', idx=9)]

We know from the data file that the two headers are in lines 1 and 19430, and we want those lines (records) from the data. Create a new dataframe called `df_idx_no_hdr` where using a sql query, you select all records but those with the header.

Before you can run a SparkSQL query, you need to "register" a dataframe.

In [0]:
# register the view
df_in_idx.createOrReplaceTempView("df_in_idx")

# run a SQL query
df_idx_no_hdr = spark.sql('select * from df_in_idx where idx !=0 and idx != 19429')

Check that the header rows were removed:

In [0]:
df_idx_no_hdr.head(10)

Out[22]: [Row(value='1 "24-HOUR MAN/EMMANUEL"', idx=1),
 Row(value='2 "3-D MAN/CHARLES CHAN"', idx=2),
 Row(value='3 "4-D MAN/MERCURIO"', idx=3),
 Row(value='4 "8-BALL/"', idx=4),
 Row(value='5 "A"', idx=5),
 Row(value='6 "A\'YIN"', idx=6),
 Row(value='7 "ABBOTT, JACK"', idx=7),
 Row(value='8 "ABCISSA"', idx=8),
 Row(value='9 "ABEL"', idx=9),
 Row(value='10 "ABOMINATION/EMIL BLO"', idx=10)]

Now you will create three separate dataframes: a `characters` one, a `publications` one, and a `relationships` one. You know the original line indices that partition the file, so use those. You have the `idx` field to use.

In [0]:
from pyspark.sql.functions import col


In [0]:
char = df_in_idx.filter(df_in_idx.idx.between(1,6486))
char.show(10)

+--------------------+---+
 value|idx|
+--------------------+---+
1 "24-HOUR MAN/EM...| 1|
2 "3-D MAN/CHARLE...| 2|
3 "4-D MAN/MERCURIO"| 3|
 4 "8-BALL/"| 4|
 5 "A"| 5|
 6 "A'YIN"| 6|
 7 "ABBOTT, JACK"| 7|
 8 "ABCISSA"| 8|
 9 "ABEL"| 9|
10 "ABOMINATION/E...| 10|
+--------------------+---+
only showing top 10 rows

In [0]:
pub = df_in_idx.filter(df_in_idx.idx.between(6487,19428))
pub.show(10)

+---------------+----+
 value| idx|
+---------------+----+
 6487 "AA2 35"|6487|
6488 "M/PRM 35"|6488|
6489 "M/PRM 36"|6489|
6490 "M/PRM 37"|6490|
 6491 "WI? 9"|6491|
 6492 "AVF 4"|6492|
 6493 "AVF 5"|6493|
 6494 "H2 251"|6494|
 6495 "H2 252"|6495|
 6496 "COC 1"|6496|
+---------------+----+
only showing top 10 rows

In [0]:
relationships = df_in_idx.filter(df_in_idx.idx.between(19430,30519))
relationships.show(10)

+--------------------+-----+
 value| idx|
+--------------------+-----+
 1 6487|19430|
2 6488 6489 6490 ...|19431|
3 6497 6498 6499 ...|19432|
 4 6506 6507 6508|19433|
 5 6509 6510 6511|19434|
6 6512 6513 6514 ...|19435|
 7 6516|19436|
 8 6517 6518|19437|
 9 6519 6520|19438|
10 6521 6522 6523...|19439|
+--------------------+-----+
only showing top 10 rows

Register the `characters` and `publications` dataframes in order to run SQL commands:

In [0]:
# registering characters and publications
char.createOrReplaceTempView("char")
pub.createOrReplaceTempView("pub")

Use the `regexp_extract` function within a SQL statement and a regular expression to create three fields from the whole line: the first field will be the integer (which is the node id), the second is the text **between** the double quotes, and the third wether it is a character or publication. Create two new separate datframes, one for characters and the other for publications.

In [0]:
char_df = spark.sql("select idx as node_id, regexp_extract(value, '^[0-9]+ \"(.*)\"$', 1) as name, \"character\" as node_type from char")

In [0]:
publication_df = spark.sql("select idx as node_id, regexp_extract(value, '^[0-9]+ \"(.*)\"$', 1) as name, \"publication\" as node_type from pub")

In [0]:
char_df.show(50)

+-------+--------------------+---------+
node_id| name|node_type|
+-------+--------------------+---------+
 1|24-HOUR MAN/EMMANUEL|character|
 2|3-D MAN/CHARLES CHAN|character|
 3| 4-D MAN/MERCURIO|character|
 4| 8-BALL/|character|
 5| A|character|
 6| A'YIN|character|
 7| ABBOTT, JACK|character|
 8| ABCISSA|character|
 9| ABEL|character|
 10|ABOMINATION/EMIL BLO|character|
 11|ABOMINATION | MUTANT|character|
 12| ABOMINATRIX|character|
 13| ABRAXAS|character|
 14| ADAM 3,031|character|
 15| ABSALOM|character|
 16|ABSORBING MAN/CARL C|character|
 17|ABSORBING MAN | MUTA|character|
 18| ACBA|character|
 19|ACHEBE, REVEREND DOC|character|
 20| ACHILLES|character|
 21| ACHILLES II/HELMUT|character|
 22| ACROBAT/CARL ZANTE|character|
 23| ADAM X|character|
 24| ADAMS, CINDY|character|
 25|ADAMS, CONGRESSMAN H|character|
 26| ADAMS, GEORGE|character|
 27| ADAMS, MARTHA|character|
 28| ADAMS, NICOLE NIKKI|character|
 29| ADAMSON, JASON|character|
 30| ADAMSON, REBECCA|character|
 31| ADMIRAL PROTOCOL/|character|
 32| ADORA|character|
 33| ADORA CLONE|character|
 34| ADRIA|character|
 35| ADVA|character|
 36| ADVENT/KYLE GROBE|character|
 37| ADVERSARY|character|
 38| AEGIS/TREY ROLLINS|character|
 39| AENTAROS|character|
 40| AFTERLIFE|character|
 41| AGAMEMNON|character|
 42|AGAMEMNON II/ANDREI |character|
 43| AGAMEMNON III/|character|
 44| AGAMOTTO|character|
 45| AGARN, CAPT.|character|
 46| AGED GENGHIS|character|
 47| AGEE, DR. AUBREY|character|
 48| AGEE, REBECCA|character|
 49| AGENT AXIS/|character|
 50|AGENT 18/JACK TRUMAN|character|
+-------+--------------------+---------+
only showing top 50 rows

In [0]:
publication_df.show(50)

+-------+--------+-----------+
node_id| name| node_type|
+-------+--------+-----------+
 6487| AA2 35|publication|
 6488|M/PRM 35|publication|
 6489|M/PRM 36|publication|
 6490|M/PRM 37|publication|
 6491| WI? 9|publication|
 6492| AVF 4|publication|
 6493| AVF 5|publication|
 6494| H2 251|publication|
 6495| H2 252|publication|
 6496| COC 1|publication|
 6497| T 208|publication|
 6498| T 214|publication|
 6499| T 215|publication|
 6500| T 216|publication|
 6501| T 440|publication|
 6502| CM 51|publication|
 6503| Q 14|publication|
 6504| Q 16|publication|
 6505| CA3 36|publication|
 6506| SLEEP 2|publication|
 6507| SLEEP 1|publication|
 6508|SLEEP 19|publication|
 6509| W2 159|publication|
 6510| W2 160|publication|
 6511| W2 161|publication|
 6512|XCAL 110|publication|
 6513|XCAL 109|publication|
 6514|XCAL 107|publication|
 6515|XCAL 108|publication|
 6516| DD/SM 1|publication|
 6517| W2 52|publication|
 6518| W2 53|publication|
 6519|XFOR 108|publication|
 6520|XFOR 109|publication|
 6521| H3 25|publication|
 6522|TTA 90/2|publication|
 6523|TTA 91/2|publication|
 6524| SS 12|publication|
 6525| T 178|publication|
 6526| H2 136|publication|
 6527| H2 137|publication|
 6528| H2 159|publication|
 6529| H2 171|publication|
 6530| H2 194|publication|
 6531| H2 195|publication|
 6532| H2 196|publication|
 6533| H2 270|publication|
 6534| H2 287|publication|
 6535| H2 278|publication|
 6536| H2 288|publication|
+-------+--------+-----------+
only showing top 50 rows

Stack both the character and publications into a single dataframe called `nodes_df`, and cache it.

In [0]:
nodes_df = char_df.union(publication_df).cache()

In [0]:
nodes_df.filter(col("node_id") > 19410).show()

+-------+--------------------+-----------+
node_id| name| node_type|
+-------+--------------------+-----------+
 19411| YC 4/4|publication|
 19412| YC 4/6|publication|
 19413| KZ3 11/2|publication|
 19414| KZ3 12/2|publication|
 19415| H' 98|publication|
 19416| SGT. FURY 8|publication|
 19417| CA: MEDUSA EFFECT|publication|
 19418|SPIDER-MAN: FEAR ITS|publication|
 19419| TOTZ 4/6|publication|
 19420| TOTZ 5|publication|
 19421| TOTZ 6|publication|
 19422| TOTZ 7|publication|
 19423| TOTZ 8|publication|
 19424| BIZADV 33|publication|
 19425| WI 25/2|publication|
 19426| AA2 30|publication|
 19427| AA2 20|publication|
 19428| AA2 38|publication|
+-------+--------------------+-----------+

Now you will work on the relationships dataframe. Import the `split` and `explode` functions:

In [0]:
from pyspark.sql.functions import split, explode

In [0]:
rel_rdd = relationships.select("value").rdd

In [0]:
relationships.show()

+--------------------+-----+
 value| idx|
+--------------------+-----+
 1 6487|19430|
2 6488 6489 6490 ...|19431|
3 6497 6498 6499 ...|19432|
 4 6506 6507 6508|19433|
 5 6509 6510 6511|19434|
6 6512 6513 6514 ...|19435|
 7 6516|19436|
 8 6517 6518|19437|
 9 6519 6520|19438|
10 6521 6522 6523...|19439|
10 6536 6537 6538...|19440|
10 6551 6552 6553...|19441|
 11 6566|19442|
 12 6567 6568 6569|19443|
13 6570 6571 6572...|19444|
14 6574 6575 6576...|19445|
15 6578 6579 6580...|19446|
16 6582 6583 6584...|19447|
16 6597 6598 6599...|19448|
16 6612 6613 6614...|19449|
+--------------------+-----+
only showing top 20 rows

In [0]:
r2 = relationships.withColumn("newcol", split(col("value"), " ")).cache() #convert value to array

Now, using a chain of daframe api methods, start with the value field in the `relationships` dataframe, wher you will create

In [0]:
r2.show()

+--------------------+-----+--------------------+
 value| idx| newcol|
+--------------------+-----+--------------------+
 1 6487|19430| [1, 6487]|
2 6488 6489 6490 ...|19431|[2, 6488, 6489, 6...|
3 6497 6498 6499 ...|19432|[3, 6497, 6498, 6...|
 4 6506 6507 6508|19433|[4, 6506, 6507, 6...|
 5 6509 6510 6511|19434|[5, 6509, 6510, 6...|
6 6512 6513 6514 ...|19435|[6, 6512, 6513, 6...|
 7 6516|19436| [7, 6516]|
 8 6517 6518|19437| [8, 6517, 6518]|
 9 6519 6520|19438| [9, 6519, 6520]|
10 6521 6522 6523...|19439|[10, 6521, 6522, ...|
10 6536 6537 6538...|19440|[10, 6536, 6537, ...|
10 6551 6552 6553...|19441|[10, 6551, 6552, ...|
 11 6566|19442| [11, 6566]|
 12 6567 6568 6569|19443|[12, 6567, 6568, ...|
13 6570 6571 6572...|19444|[13, 6570, 6571, ...|
14 6574 6575 6576...|19445|[14, 6574, 6575, ...|
15 6578 6579 6580...|19446|[15, 6578, 6579, ...|
16 6582 6583 6584...|19447|[16, 6582, 6583, ...|
16 6597 6598 6599...|19448|[16, 6597, 6598, ...|
16 6612 6613 6614...|19449|[16, 6612, 6613, ...|
+--------------------+-----+--------------------+
only showing top 20 rows

In [0]:
edges_df = relationships.withColumn("newcol", split(col("value"), " ")) \
    .withColumn("character", col("newcol").getItem(0)) \
    .select("character", explode(col("newcol")).alias("publication")) \
    .filter(col("character") != col("publication")).cache()

In [0]:
edges_df.show(50)

+---------+-----------+
character|publication|
+---------+-----------+
 1| 6487|
 2| 6488|
 2| 6489|
 2| 6490|
 2| 6491|
 2| 6492|
 2| 6493|
 2| 6494|
 2| 6495|
 2| 6496|
 3| 6497|
 3| 6498|
 3| 6499|
 3| 6500|
 3| 6501|
 3| 6502|
 3| 6503|
 3| 6504|
 3| 6505|
 4| 6506|
 4| 6507|
 4| 6508|
 5| 6509|
 5| 6510|
 5| 6511|
 6| 6512|
 6| 6513|
 6| 6514|
 6| 6515|
 7| 6516|
 8| 6517|
 8| 6518|
 9| 6519|
 9| 6520|
 10| 6521|
 10| 6522|
 10| 6523|
 10| 6524|
 10| 6525|
 10| 6526|
 10| 6527|
 10| 6528|
 10| 6529|
 10| 6530|
 10| 6531|
 10| 6532|
 10| 6533|
 10| 6534|
 10| 6535|
 10| 6536|
+---------+-----------+
only showing top 50 rows

In [0]:
#  Register relationships as a temporary view
nodes_df.createOrReplaceTempView("nodes_df")
edges_df.createOrReplaceTempView("edges_df")

In [0]:
spark.sql("select count(*) from edges_df").collect()

Out[70]: [Row(count(1)=96662)]

## Create an analytical dataset

You will now create an analytical dataset using SparkSQL where you will join both tables (nodes_df and edge_df) so you have the data you need to run some analytics on the data.

In [0]:
edges_df.show(10)

+---------+-----------+
character|publication|
+---------+-----------+
 1| 6487|
 2| 6488|
 2| 6489|
 2| 6490|
 2| 6491|
 2| 6492|
 2| 6493|
 2| 6494|
 2| 6495|
 2| 6496|
+---------+-----------+
only showing top 10 rows

In [0]:
nodes_df.show(10)

+-------+--------------------+---------+
node_id| name|node_type|
+-------+--------------------+---------+
 1|24-HOUR MAN/EMMANUEL|character|
 2|3-D MAN/CHARLES CHAN|character|
 3| 4-D MAN/MERCURIO|character|
 4| 8-BALL/|character|
 5| A|character|
 6| A'YIN|character|
 7| ABBOTT, JACK|character|
 8| ABCISSA|character|
 9| ABEL|character|
 10|ABOMINATION/EMIL BLO|character|
+-------+--------------------+---------+
only showing top 10 rows

In [0]:
analytics = spark.sql("""select e.character as character_id, c.name as character, 
                      e.publication as publication_id, p.name as publication
                      from edges_df e 
                      left join nodes_df c on e.character = c.node_id
                      left join nodes_df p on e.publication = p.node_id
                      """).cache()


In [0]:
analytics.show(10)

+------------+--------------------+--------------+-----------+
character_id| character|publication_id|publication|
+------------+--------------------+--------------+-----------+
 1|24-HOUR MAN/EMMANUEL| 6487| AA2 35|
 2|3-D MAN/CHARLES CHAN| 6488| M/PRM 35|
 2|3-D MAN/CHARLES CHAN| 6489| M/PRM 36|
 2|3-D MAN/CHARLES CHAN| 6490| M/PRM 37|
 2|3-D MAN/CHARLES CHAN| 6491| WI? 9|
 2|3-D MAN/CHARLES CHAN| 6492| AVF 4|
 2|3-D MAN/CHARLES CHAN| 6493| AVF 5|
 2|3-D MAN/CHARLES CHAN| 6494| H2 251|
 2|3-D MAN/CHARLES CHAN| 6495| H2 252|
 2|3-D MAN/CHARLES CHAN| 6496| COC 1|
+------------+--------------------+--------------+-----------+
only showing top 10 rows

What are the top 10 publications with the highest number of characters?

In [0]:
analytics.createOrReplaceTempView("analytics")
spark.sql("""select publication, count(*) as char_ct from analytics 
            group by publication order by char_ct desc limit 10""").show()

+--------------------+-------+
 publication|char_ct|
+--------------------+-------+
 COC 1| 111|
MARVEL MYSTERY COMIC| 92|
 IW 3| 91|
 IW 1| 90|
 H2 279| 87|
 IW 4| 80|
 IW 2| 76|
 MAXSEC 3| 72|
 FF 370| 63|
 IW 6| 60|
+--------------------+-------+

## Saving a DataFrame to a csv

In [0]:
analytics.write\
    .format("com.databricks.spark.csv")\
    .option("header", "true")\
    .save("path-in-hdfs-or-cloud")

This lab is based on [https://vincentlauzon.com/2018/01/24/azure-databricks-spark-sql-data-frames/](https://vincentlauzon.com/2018/01/24/azure-databricks-spark-sql-data-frames/). You can see the code in that blog post to perform the same operations using RDD commands.